# 知识图谱数据调用探索

- 参考:
    - https://kq4b3vgg5b.feishu.cn/wiki/MCs6wDHyMiAJp5kHVv7cKsnqnLf?table=tbl8Hhg2lpLG6EM0&view=vewDvZhc5C
    - https://blog.csdn.net/2401_85378759/article/details/144713235
    - https://blog.csdn.net/Andy_shenzl/article/details/145590829

In [1]:
import os
import pandas as pd

In [2]:
import tiktoken

from graphrag.query.context_builder.entity_extraction import EntityVectorStoreKey
from graphrag.query.indexer_adapters import (
    read_indexer_covariates,
    read_indexer_entities,
    read_indexer_relationships,
    read_indexer_reports,
    read_indexer_text_units,
)

from graphrag.query.question_gen.local_gen import LocalQuestionGen
from graphrag.query.structured_search.local_search.mixed_context import (
    LocalSearchMixedContext,
)
from graphrag.query.structured_search.local_search.search import LocalSearch
from graphrag.vector_stores.lancedb import LanceDBVectorStore

In [3]:
INPUT_DIR = "../graphRag/output"
LANCEDB_URI = f"{INPUT_DIR}/lancedb"

COMMUNITY_REPORT_TABLE = "community_reports"
COMMUNITY_TABLE = "communities"
ENTITY_TABLE = "entities"
DOCUMENT_TABLE = "documents"
RELATIONSHIP_TABLE = "relationships"
TEXT_UNIT_TABLE = "text_units"
COMMUNITY_LEVEL = 2

## 实体

In [4]:
entity_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_TABLE}.parquet")
print(f"Entity count: {len(entity_df)}")
entity_df.head(3)

Entity count: 1021


,id,human_readable_id,title,type,description,text_unit_ids,frequency,degree,x,y
0,7d8dfeeb-d5f6-4f25-ae86-f69c7994249e,0,绍兴,GEO,绍兴是中国浙江省的一个历史名城，宋代时期就已经非常重要。李清照，作为宋代著名的女词人，她的生...,[ed0d7a7cec224017e5cfe7df292b6d6a031967471fba1...,4,7,0.0,0.0
1,73125e53-678d-4ca9-b526-7679bf230aec,1,高宗,PERSON,宋高宗赵构，南宋初期的皇帝，同时也是宋真宗的第九子。在金军入侵北宋后期和南宋初期，国家动荡不...,[ed0d7a7cec224017e5cfe7df292b6d6a031967471fba1...,9,9,0.0,0.0
2,615e99dc-26e7-4a59-b424-b628ea596305,2,1129年,DATE,1129年，即建炎三年，李清照46岁。这一年对她来说意义重大，不仅因为她在这一年创作了晚期的...,[ed0d7a7cec224017e5cfe7df292b6d6a031967471fba1...,5,8,0.0,0.0


In [5]:
entity_embedding_df = pd.read_parquet(f"{INPUT_DIR}/{COMMUNITY_TABLE}.parquet")
print(f"Entity count: {len(entity_embedding_df)}")
entity_embedding_df.head(3)

Entity count: 128


,id,human_readable_id,community,level,parent,children,title,entity_ids,relationship_ids,text_unit_ids,period,size
0,9eb41439-c5d3-4cde-9627-2395f12776d9,0,0,0,-1,"[40, 41, 42, 43, 44, 45, 46, 47, 48]",Community 0,"[fa0fdf70-4d48-4f9a-a0d6-156ebd898e4d, e935cc0...","[0059ae6c-34ce-49c1-b796-76f09638686d, 040d1e7...",[02b3e0c7578ca98f4ade399261fe7f80bd1515a253117...,2025-03-28,272
1,62ae8873-9aa1-4668-b8b4-546c00aa1b90,1,1,0,-1,[],Community 1,"[d339717a-607a-4b01-83a6-3f9ebb6ac851, 966b71a...","[190f6382-aadb-476d-9038-7b92658eee4b, c6ec38b...",[cc3ca7cf89d791f5245c6eace9b945ae9d04a805fec2b...,2025-03-28,3
2,b6458935-a64e-4182-b08e-599ebe900bea,2,2,0,-1,[],Community 2,"[7748185d-cf93-4df8-995d-65f10bd45da3, 72612cd...","[0d0adf0d-c0ff-4d46-8092-12caf5cb7ae1, 6aa1366...",[63006da44a5bc1180ed270ca98b03db18891c028fe6d3...,2025-03-28,3


In [6]:
entities = read_indexer_entities(entity_df, entity_embedding_df, COMMUNITY_LEVEL)
entities[:3]

[Entity(id='006872d9-58ba-4f89-a9ce-40e69b382ffd', short_id='994', title='梧桐树', type='EVENT', description='在诗中常用来象征秋天和离愁别绪的树木', description_embedding=None, name_embedding=None, community_ids=['34', '110'], text_unit_ids=['7f32a0860d5cbbfd6a170edfad5cde3369f292d95925658541c013279825eaab075ce1b7f944c67e18cbb6aad503e91395a44236f8407fb8de710b034f8f8106'], rank=3, attributes=None),
 Entity(id='0133c5a3-aa3e-474c-a28a-531a261937d3', short_id='805', title='中秋', type='EVENT', description='中国传统节日，桂花在此时节开放，被李清照称为花中之冠', description_embedding=None, name_embedding=None, community_ids=['34', '109'], text_unit_ids=['f44a1440a909266504e060ecd6b8f3998076e48084e3897de1131a0c88cc443de254a854a17aaae039ede00c11ff973c73d87bf21e4e1340cca17c1e28218b0e'], rank=1, attributes=None),
 Entity(id='013f2f31-bf4e-4c3b-bcb7-e3a4ba835f10', short_id='228', title='建康城', type='GEO', description='建康城，即古代的秣陵，是李清照生活和创作的重要地点之一。1128年12月，这位宋代女诗人曾登上建康城寻找诗歌的灵感。建康城不仅在李清照的生命轨迹中占据了一席之地，还与整个南宋历史背景紧密相连，见证了那个时代的文化与社会动态。李清照在那里的探访也反映了宋代知

## 关系

In [7]:
relationship_df = pd.read_parquet(f"{INPUT_DIR}/{RELATIONSHIP_TABLE}.parquet")
relationships = read_indexer_relationships(relationship_df)
print(f"Relationship count: {len(relationship_df)}")
relationship_df.head(3)

Relationship count: 1138


,id,human_readable_id,source,target,description,weight,combined_degree,text_unit_ids
0,2a5f9d86-1755-46cc-adc8-f7d78b99284e,0,绍兴,1129年,李清照在1129年追随高宗途中居住于绍兴,2.0,15,[ed0d7a7cec224017e5cfe7df292b6d6a031967471fba1...
1,6a68643a-acb6-4a9b-ac6f-d4a00a3d1c19,1,高宗,李清照,李清照，南宋著名女词人，词风婉约，被誉为“词中的女杰”。在1129年至1130年间，李清照经...,6.0,472,[ed0d7a7cec224017e5cfe7df292b6d6a031967471fba1...
2,15e689c4-27aa-453d-8243-dacd41d5d158,2,1129年,《清平乐》,李清照在1129年创作了反映个人心路历程的词《清平乐》,2.0,11,[ed0d7a7cec224017e5cfe7df292b6d6a031967471fba1...


In [8]:
relationships[:3]

[Relationship(id='2a5f9d86-1755-46cc-adc8-f7d78b99284e', short_id='0', source='绍兴', target='1129年', weight=2.0, description='李清照在1129年追随高宗途中居住于绍兴', description_embedding=None, text_unit_ids=['ed0d7a7cec224017e5cfe7df292b6d6a031967471fba15766cc24f3e2a6ea1938a39a6e1ade1469d75968ff39c54f128bdb85fd50969224885963128a6a43933'], rank=15, attributes=None),
 Relationship(id='6a68643a-acb6-4a9b-ac6f-d4a00a3d1c19', short_id='1', source='高宗', target='李清照', weight=6.0, description='李清照，南宋著名女词人，词风婉约，被誉为“词中的女杰”。在1129年至1130年间，李清照经历了一段颠沛流离的生活。据史料记载，她跟随当时的南宋君主高宗，也因金兵入侵南迁，至两浙一带。在1130年2月，她仍在追随高宗在浙东地区辗转流亡。这一时期，她所见所感都深深影响着她的作品，使其词作充满了对失去的家园的深切怀念和对动荡岁月的感慨。这一段经历不仅展示了其坚韧不拔的个性，也为研究李清照的词作提供了重要的历史背景。', description_embedding=None, text_unit_ids=['ed0d7a7cec224017e5cfe7df292b6d6a031967471fba15766cc24f3e2a6ea1938a39a6e1ade1469d75968ff39c54f128bdb85fd50969224885963128a6a43933', 'ca7bfa460129b5b6c75a6ef3f29eada7ffe11870437687ff65e703ff00bf54307f66033d716c0dd728ddb9a1634ddfdfd8d2e4f66c62637e1841137af9b27158', 'a271f2f2

## 报告

In [9]:
report_df = pd.read_parquet(f"{INPUT_DIR}/{COMMUNITY_REPORT_TABLE}.parquet")

print(f"Report records: {len(report_df)}")
report_df.head(3)

Report records: 122


,id,human_readable_id,community,level,parent,children,title,summary,full_content,rank,rating_explanation,findings,full_content_json,period,size
0,455bee04dbf746b68cea169ac87f5836,112,112,2,69,[],李清照与赵明诚的金石学研究,本报告围绕宋代著名女诗人李清照及其丈夫赵明诚的金石学研究，特别是他们合著的《金石录》以及与文...,# 李清照与赵明诚的金石学研究\n\n本报告围绕宋代著名女诗人李清照及其丈夫赵明诚的金石学研...,7.0,赵明诚与李清照在金石学研究和文学方面的成就，对中国历史文化有深远影响。,[{'explanation': '《金石录》是由李清照和赵明诚共同整理，其记录了大量的金石...,"{\n ""title"": ""李清照与赵明诚的金石学研究"",\n ""summary...",2025-03-28,6
1,9f2fff76dba04d34abd944f0bfa743cd,113,113,2,69,[],归来堂与1115年的文化交流,归来堂是李清照与赵明诚学术交流与合作的关键地点，1115年象征着这对夫妇在艺术与学问上的共同...,# 归来堂与1115年的文化交流\n\n归来堂是李清照与赵明诚学术交流与合作的关键地点，11...,2.0,虽然这对夫妇对金石学的贡献被历史所记住，但其影响范围相对较小，主要局限于学术界。,[{'explanation': '1115年是李清照的个人艺术成就的标志性年份，她创作了著...,"{\n ""title"": ""归来堂与1115年的文化交流"",\n ""summar...",2025-03-28,2
2,72ee3526d41b4e3f8fbbe32485c29544,114,114,2,69,[],江宁与李清照的历史联系,江宁作为历史著名的古都，在文人李清照的生活与作品中占据了重要的地位。李清照在1128年2月到...,# 江宁与李清照的历史联系\n\n江宁作为历史著名的古都，在文人李清照的生活与作品中占据了重...,4.0,江宁与李清照之间的联系在中国历史和文化上具有一定的重要性。,[{'explanation': '江宁既是中国历史上的重要古都，也是许多文人墨客的创作寄托...,"{\n ""title"": ""江宁与李清照的历史联系"",\n ""summary"":...",2025-03-28,2


In [10]:
reports = read_indexer_reports(report_df, entity_embedding_df, COMMUNITY_LEVEL)
reports[:3]

[CommunityReport(id='455bee04dbf746b68cea169ac87f5836', short_id='112', title='李清照与赵明诚的金石学研究', community_id='112', summary='本报告围绕宋代著名女诗人李清照及其丈夫赵明诚的金石学研究，特别是他们合著的《金石录》以及与文人陆德夫的文学交流。此外，赵明诚在1129年前往行在的个人经历也被纳入考量。', full_content='# 李清照与赵明诚的金石学研究\n\n本报告围绕宋代著名女诗人李清照及其丈夫赵明诚的金石学研究，特别是他们合著的《金石录》以及与文人陆德夫的文学交流。此外，赵明诚在1129年前往行在的个人经历也被纳入考量。\n\n## 《金石录》的学术价值\n\n《金石录》是由李清照和赵明诚共同整理，其记录了大量的金石拓片及其解读，反映了二人对金石文物的深刻理解和广泛收集整理工作[数据：实体（391），关系（563）]。\n\n## 李清照的文学与金石学成就\n\n李清照不仅在诗词歌赋方面有卓越成就，同时也在文物研究与历史文化保存方面做出了贡献，这体现在她合著的《金石录》以及与陆德夫的文学交流中[数据：实体（391），关系（932）]。\n\n## 赵明诚与陆德夫的文学交流\n\n赵明诚与陆德夫之间的关系突显了宋代文学社群的交流模式。赵明诚不仅赞赏李清照的创作，还将她的词与自身的作品融合后呈现给陆德夫[数据：实体（860），关系（932）]。\n\n## 赵明诚在1129年的经历\n\n在1129年，赵明诚离开江宁，并在6月13日前往建康，这表明了他个人与行政事务的关联[数据：实体（919，311），关系（988，351，992）]。\n\n## 赵明诚与李清照的深刻连接\n\n无论是在专业的金石学合作还是日常生活的点滴中，赵明诚与李清照之间的关系深深植根于他们的作品和情感生活中[数据：实体（391，148），关系（563，168）]。', rank=7.0, full_content_embedding=None, attributes=None, size=None, period=None),
 CommunityReport(id='9f2fff76dba04d34abd944f0bfa743cd', short_id='113',

## 文本块

In [11]:
text_unit_df = pd.read_parquet(f"{INPUT_DIR}/{TEXT_UNIT_TABLE}.parquet")

print(f"Text unit records: {len(text_unit_df)}")
text_unit_df.head(3)

Text unit records: 100


,id,human_readable_id,text,n_tokens,document_ids,entity_ids,relationship_ids,covariate_ids
0,ed0d7a7cec224017e5cfe7df292b6d6a031967471fba15...,1,绍兴（1129年，46岁，作品：1）\n1129年，46岁\n1129年 追随高宗于两浙，有...,1200,[03e4742412e1be85fbc7a7737fe0cc9da4241d2ce5e76...,"[7d8dfeeb-d5f6-4f25-ae86-f69c7994249e, 73125e5...","[2a5f9d86-1755-46cc-adc8-f7d78b99284e, 6a68643...",[]
1,ca7bfa460129b5b6c75a6ef3f29eada7ffe11870437687...,2,世的艰辛。当年那种赏梅的雅兴大减。这两句写的就是词人婚后的这段生活，表现的是一种百无聊赖、忧...,1027,[03e4742412e1be85fbc7a7737fe0cc9da4241d2ce5e76...,"[7d8dfeeb-d5f6-4f25-ae86-f69c7994249e, 73125e5...","[6a68643a-acb6-4a9b-ac6f-d4a00a3d1c19, feacc67...",[]
2,35db9c01730e44b834f55c18ffe1db83515f95c656b2d9...,3,7月 金人立刘豫为帝，清照作诗斥伪齐。作品：《咏史》\n\n绍兴（1131年3月 - 113...,127,[03e4742412e1be85fbc7a7737fe0cc9da4241d2ce5e76...,"[7d8dfeeb-d5f6-4f25-ae86-f69c7994249e, 73125e5...","[e6c0a66e-70f9-44da-bb35-ddc48882bcb8, 796f3ca...",[]


In [12]:
text_units = read_indexer_text_units(text_unit_df)
text_units[:3]

[TextUnit(id='ed0d7a7cec224017e5cfe7df292b6d6a031967471fba15766cc24f3e2a6ea1938a39a6e1ade1469d75968ff39c54f128bdb85fd50969224885963128a6a43933', short_id='0', text='绍兴（1129年，46岁，作品：1）\n1129年，46岁\n1129年\u3000追随高宗于两浙，有词写奔亡之苦。作品：《清平乐》（年年雪里）\n\n清平乐（1129年） 宋 · 李清照\n年年雪里，常插梅花醉，挼尽梅花无好意，赢得满衣清泪！\n今年海角天涯，萧萧两鬓生华。\n看取晚来风势，故应难看梅花。\n【评注】\n这是一首典型的赏梅词作，借不同时期的赏梅感爱写出了词人个人的心路历程：少年的欢乐，中年的幽怨，晚年的沧落，在词中都约略可见。词意含蓄蕴藉，感情悲切哀婉，以赏梅寄寓自己的今苦之感和永国之忧，感慨深沉。\n上阕回忆往昔赏梅生活，分为两层。“年年雪里，常插梅花醉 。”这两句抓住富有特征的生活细节生动地再现了词人早年赏梅的情景和兴致，表现出少女的纯真 、欢乐和闲适。她早年写下的咏梅词《渔家傲》中有句云 ：“雪里已知春信至，寒梅点缀琼枝腻⋯⋯共赏金尊沉绿蚁，莫辞醉，此花不与群花比 。”可作为“年年雪里，常插梅花醉”的注脚。接下来“挼尽梅花无好意 ，赢得满衣清泪”两句，心绪显然不同，虽然梅枝在手，却无好心情去赏玩，只是漫不经心地揉搓着。赏梅原本为的是排遣心头的忧伤，可是本来心情就不好，到头来不仅忧伤没有消除，反倒触景生情，激起更深的伤感，落得个“满衣清泪 ”。花还是昔日的花，然而花相似，人不同，物是人非，怎不使人伤心落泪呢？李清照婚后，夫妻志同道合、伉俩相得，生活美满幸福。但是，时常发生的短暂离别使她识尽离愁别苦。在婚后六、七年的时间里，李赵两家相继罹祸，紧接着就开始了长期的“屏居乡里”的生活。生活的坎坷使她屡处忧患，饱尝人世的艰辛。当年那种赏梅的雅兴大减。这两句写的就是词人婚后的这段生活，表现的是一种百无聊赖、忧伤怨恨的情绪。\n上阕四句回忆了作者两个生活阶段赏梅时的又一不同情景和心情。\n下阕以“今年”两字领起，同上阕的“年年”相对 。往年是“常插梅花醉”；即使是“挼尽梅花无好意”的时候，也多半为的是离别相思。眼前却截然不同了，“今

In [13]:
import requests
from typing import Any, Dict, List

class OpenaiApiType:
    """
    Enum-like class to define API types.
    """
    OpenAI = "openai"
    Azure = "azure"
    Custom = "custom"

class OpenAIEmbedding:
    def __init__(
        self,
        api_key: str,
        api_base: str,
        api_type: str,
        model: str,
        max_retries: int = 3,
    ):
        self.api_key = api_key
        self.api_base = api_base.rstrip("/")  # Ensure no trailing slash
        self.api_type = api_type
        self.model = model
        self.max_retries = max_retries

    def embed(self, text: str, **kwargs: Any) -> List[float]:
        url = f"{self.api_base}/embeddings"
        payload = {
            "model": self.model,
            "input": text,
            **kwargs,
        }
        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {self.api_key}",
        }

        retries = 0
        while retries < self.max_retries:
            try:
                response = requests.post(url, json=payload, headers=headers)
                response.raise_for_status()  # Raise an exception for HTTP errors
                data = response.json()
                return data.get("data", [])[0].get("embedding", [])
            except requests.exceptions.RequestException as e:
                retries += 1
                if retries >= self.max_retries:
                    raise RuntimeError(f"Failed to generate embedding after {self.max_retries} retries: {e}")
                print(f"Retrying... ({retries}/{self.max_retries})")


In [14]:
import tiktoken
token_encoder = tiktoken.get_encoding("cl100k_base")

text_embedder = OpenAIEmbedding(
    api_key='ollama',
    api_base="http://localhost:11434/v1",
    api_type=OpenaiApiType.OpenAI,
    model='mxbai-embed-large',
    max_retries=20,
)

In [15]:
text = "李清照你的父亲是谁"
embedding = text_embedder.embed(text)
print(f"Embedding: {embedding}")

Retrying... (1/20)
Retrying... (2/20)
Retrying... (3/20)
Retrying... (4/20)
Retrying... (5/20)
Retrying... (6/20)
Retrying... (7/20)
Retrying... (8/20)
Retrying... (9/20)
Retrying... (10/20)
Retrying... (11/20)
Retrying... (12/20)


KeyboardInterrupt: 

In [70]:
description_embedding_store = LanceDBVectorStore(
    collection_name="default-entity-description",
)
description_embedding_store.connect(db_uri=LANCEDB_URI)

In [71]:
context_builder = LocalSearchMixedContext(
    entities=entities,
    entity_text_embeddings=description_embedding_store,
    text_embedder=text_embedder,
    text_units=text_units,
    community_reports=reports,
    relationships=relationships,
    covariates=None,
    token_encoder=token_encoder,
    embedding_vectorstore_key=EntityVectorStoreKey.ID,
)

site-packages\graphrag\vector_stores\lancedb.py



```py
def similarity_search_by_vector(
    self, query_embedding: list[float], k: int = 10, **kwargs: Any
) -> list[VectorStoreSearchResult]:
    """Perform a vector-based similarity search."""
    if self.query_filter:
        docs = (
            self.document_collection.search(
                query=query_embedding, vector_column_name="vector"
            )
            .where(self.query_filter, prefilter=True)
            .limit(k)
            .to_list()
        )
    else:
        docs = (
            self.document_collection.search(
                query=query_embedding, vector_column_name="vector"
            )
            .limit(k)
            .to_list()
        )
    return [
        VectorStoreSearchResult(
            document=VectorStoreDocument(
                id=doc["id"],
                text=doc["text"],
                vector=doc["vector"],
                attributes=json.loads(doc["attributes"]),
            ),
            score=1 - abs(float(doc["_distance"])),
        )
        for doc in docs if abs(float(doc["_distance"]))<0.3 # 修改这里
    ]
```

## 测试效果

### 存在的实体匹配效果展示

In [94]:
# test1
text1 = "赵明诚"
context1 = context_builder.build_context(text1)

In [95]:
chunks1 = context1.context_chunks
records1 = context1.context_records

In [96]:
print(type(records1))
print("*"*100)
print(records1.keys())
print("*"*100)
print('reports:')
print(records1['reports'])
print("*"*100)
print('relationships:')
print(records1['relationships'])
print("*"*100)
print('entities:')
print(records1['entities'])
print("*"*100)
print('sources:')
print(records1['sources'])

<class 'dict'>
****************************************************************************************************
dict_keys(['reports', 'relationships', 'entities', 'sources'])
****************************************************************************************************
reports:
   id          title                                            content
0   0     李清照与宋代文化社区  # 李清照与宋代文化社区\n\n本报告聚焦于宋代女词人李清照及其作品与生活环境的关系，探讨了...
1  30  李清照与宋代文人的文学交融  # 李清照与宋代文人的文学交融\n\n本报告围绕宋代杰出女词人李清照及与她有显著文学互动的欧...
****************************************************************************************************
relationships:
     id source target                                        description  \
0   406     明诚    李清照  李清照是中国宋代著名的女词人，她的才华和贡献在中国文学史上占有重要地位。她的丈夫名为明诚，也...   
1   166    李清照     明诚  李清照是中国宋代著名的女词人，与丈夫明诚的情感深厚，两人不仅情感生活融洽，而且在学术工作上也...   
2   384     明诚     易安                               李清照（易安）与赵明诚（明诚）的夫妻关系   
3   933     易安     明诚    李清照创作词作后，赵明诚感到赞叹，并试图创作更佳作品\n易安李清照作词，明诚赵明诚对此作词赞叹   

In [90]:
type(records1['reports']),type(records1['relationships']),type(records1['entities']),type(records1['sources'])

(pandas.core.frame.DataFrame,
 pandas.core.frame.DataFrame,
 pandas.core.frame.DataFrame,
 pandas.core.frame.DataFrame)

In [84]:
print(type(chunks1))
print(chunks1)

<class 'str'>
-----Reports-----
id|title|content
0|李清照与宋代文化社区|"# 李清照与宋代文化社区

本报告聚焦于宋代女词人李清照及其作品与生活环境的关系，探讨了她的词作如何反映了宋代的文化氛围和个人情感。李清照的作品与多个实体有关，包括个人经历、创作背景和文化符号。

## 李清照的文学成就与宋代文化的关联

李清照是宋代著名的女词人，其作品深受当时社会文化的影响，同时也反映了个人的情感和生活体验。她的词作中融入了许多文化元素和个人情感，如《一剪梅》中的玉簟象征，以及对陶渊明等历史人物的提及，显示了她与宋代文化的紧密联系[数据：实体（6，84，331，100）；关系（105，375，117）]。

## 李清照作品中的个人情感表达

李清照的词作不仅展现了她对宋代文化的深刻理解，还体现了她个人情感的细腻表达。例如，她在《念奴娇 春情》中对萧条庭院和斜风细雨的描写，传达了深深的愁绪和对过去时光的怀念[数据：实体（123，124）；关系（141，142）]。

## 李清照与宋代社会生活的反映

通过对李清照作品的分析，我们可以看到她的生活及创作受到了宋代文化氛围的深刻影响。她的词作中的物质文化细节，如玉簟、金翠罗衣等，不仅描绘了当时社会的繁荣，也反映了个人生活的物质基础[数据：实体（331，78）；关系（375，98）]。

## 李清照词作中的历史与神话元素

李清照的词作中融入了丰富的历史与神话元素，如织女、韩令等，这些元素不仅增加了作品的文化深度，也使其成为研究宋代文化和文学的重要资料[数据：实体（721，81）；关系（766，102）]。

## 李清照与宋代政治变迁的关系

李清照的生活和创作受到了宋代政治变迁的影响。例如，朝廷放散百官后，她移居衢州，这一事件及其对应的作品，如《咏史》中的斥责刘豫为伪齐帝，反映了她对时政的关注和态度[数据：实体（921，923）；关系（993，998，17）]。

## 李清照词作中的自然景观描写

李清照的词作中经常出现对自然景观的描写，如清露、新桐等，这些描写不仅展现了她对自然美的敏感和欣赏，也与她的情感体验紧密相连，如春寒与内心愁绪的联系[数据：实体（166，167）；关系（186，187，181）]。"
30|李清照与宋代文人的文学交融|"# 李清照与宋代文人的文

### 不存在的实体匹配效果展示

In [100]:
# test2
text2 = "中南大学"
context2 = context_builder.build_context(text2)
chunks2 = context2.context_chunks
records2 = context2.context_records

In [101]:
print(records2['reports'])
print("*" * 100)
print('relationships:')
print(records2['relationships'])
print("*" * 100)
print('entities:')
print(records2['entities'])
print("*" * 100)
print('sources:')
print(records2['sources'])

   id       title                                            content
0  11    李清照与宋代词坛  # 李清照与宋代词坛\n\n社区围绕李清照，宋代著名女词人展开，其作品及别号易安影响深远。该...
1  16  李清照与宋代文化环境  # 李清照与宋代文化环境\n\n本社区以李清照及其关联实体构成，体现了宋代文化环境下李清照诗...
****************************************************************************************************
relationships:
     id source target                                        description  \
0   792     秦楼    李清照                                李清照独自在秦楼凝望，寓意着深切的思念   
1   781    李清照     秦楼                      李清照词作中提到的秦楼，可能象征着别离或者是心情愁绪的地方   
2    60    李清照     庄子                          李清照在作品中引用庄子的《逍遥游》来扩展自己的想象   
3   832    婉约派     易安                          易安的词作体现了正宗的婉约派风格，清新浅易而不攲靡   
4   723    八咏楼     沈约  八咏楼是根据沈约的《八咏》诗集而命名的地方。沈约不仅建造了八咏楼，还在此赋诗，可见他与这座楼...   
5    65     庄子     词人                        词人在作品中引用庄子的《逍遥游》，描述大鹏鸟的宏大形象   
6    86     格非    太学录                                      格非在1086年进补太学录   
7   793     秦楼    阳关曲                          秦楼作为凝望之地，与《阳关曲》中的离愁别绪形